In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import tkinter as tk
from tkinter import simpledialog, messagebox, ttk
import threading

In [2]:
dfre = pd.read_csv('cartelera.csv')

In [3]:
gui_finished_event = threading.Event()

def obtener_valoraciones():
    valoraciones = {}
    while True:
        # Crear una ventana de diálogo personalizada con autocompletar
        dialog = AutocompleteDialog(root, dfre['title'].values)
        root.wait_window(dialog.top)
        
        title = dialog.result
        if not title:
            break
        # Verificar si el título está en la cartelera
        if title not in dfre['title'].values:
            messagebox.showerror("Error", f"La película '{title}' no está en nuestra cartelera.")
            continue
        # Preguntar al usuario por la valoración de la película
        rating = simpledialog.askfloat("Input", f"Introduce la valoración para '{title}':")
        if rating is not None:
            valoraciones[title] = rating
    return valoraciones

def mostrar_resultado(df):
    if df.empty:
        messagebox.showinfo("Información", "No hay datos de películas valoradas para mostrar.")
    else:
        result_window = tk.Toplevel(root)
        result_window.title("Películas Valoradas")
        text = tk.Text(result_window)
        text.pack(expand=True, fill='both')
        text.insert(tk.END, df.to_string(index=False))

def main():
    global df_valoraciones_completas
    valoraciones = obtener_valoraciones()
    print("Valoraciones obtenidas:", valoraciones)  # Mensaje de diagnóstico
    if valoraciones:
        # Convertir las valoraciones a un DataFrame
        df_nuevas_valoraciones = pd.DataFrame(list(valoraciones.items()), columns=['title', 'Rating'])
        print("DataFrame de nuevas valoraciones creado:\n", df_nuevas_valoraciones)  # Mensaje de diagnóstico
        # Unir con dfre para obtener todos los datos de las películas valoradas
        df_valoraciones_completas = pd.merge(left=df_nuevas_valoraciones, right=dfre, left_on='title', right_on='title', how='inner')
        print("DataFrame de valoraciones completas creado:\n", df_valoraciones_completas)  # Mensaje de diagnóstico
        mostrar_resultado(df_valoraciones_completas)
    else:
        messagebox.showinfo("Información", "No se introdujeron valoraciones.")
    # Señalar que la GUI ha terminado
    gui_finished_event.set()

def run_gui():
    global root
    root = tk.Tk()
    root.withdraw()  # Esconder la ventana principal
    main()
    root.mainloop()

class AutocompleteDialog:
    def __init__(self, parent, suggestions):
        top = self.top = tk.Toplevel(parent)
        self.top.title("Introduce el título de la película")
        self.suggestions = suggestions
        self.result = None
        
        self.label = tk.Label(top, text="Introduce el título de la película (o deja vacío para terminar):")
        self.label.pack(side="top", fill="x", padx=20, pady=5)
        
        self.entry_var = tk.StringVar()
        self.entry = ttk.Entry(top, textvariable=self.entry_var)
        self.entry.pack(side="top", fill="x", padx=20, pady=5)
        
        self.listbox = tk.Listbox(top)
        self.listbox.pack(side="top", fill="both", expand=True, padx=20, pady=5)
        
        self.entry_var.trace("w", self.update_listbox)
        self.entry.bind("<Return>", self.on_select)
        self.listbox.bind("<Double-Button-1>", self.on_select)
        
        self.entry.focus_set()

    def update_listbox(self, *args):
        search_term = self.entry_var.get()
        self.listbox.delete(0, tk.END)
        for item in self.suggestions:
            if search_term.lower() in item.lower():
                self.listbox.insert(tk.END, item)
    
    def on_select(self, event=None):
        try:
            index = self.listbox.curselection()[0]
            self.result = self.listbox.get(index)
        except IndexError:
            self.result = self.entry_var.get()
        self.top.destroy()

# Ejecutar la interfaz gráfica en un hilo separado
thread = threading.Thread(target=run_gui)
thread.start()

# Esperar hasta que la GUI termine
gui_finished_event.wait()

# Aquí puedes continuar con otras celdas del notebook
print("La interfaz gráfica ha terminado. Continuando con las siguientes celdas del notebook.")

# Mostrar el DataFrame de valoraciones completas para asegurarnos de que está disponible
if 'df_valoraciones_completas' in globals():
    print(df_valoraciones_completas.head())
else:
    print("No hay valoraciones completas disponibles.")

Valoraciones obtenidas: {'The Matrix': 3.0, 'The Lord of the Rings: The Two Towers': 5.0, 'The Lord of the Rings: The Fellowship of the Ring': 4.0, 'Star Wars': 2.0, 'Home Alone': 2.0, 'Thor: Ragnarok': 1.0, 'Game of Thrones': 4.0, 'Vikings': 4.0, 'Boarding School': 3.0}
DataFrame de nuevas valoraciones creado:
                                                title  Rating
0                                         The Matrix     3.0
1              The Lord of the Rings: The Two Towers     5.0
2  The Lord of the Rings: The Fellowship of the Ring     4.0
3                                          Star Wars     2.0
4                                         Home Alone     2.0
5                                     Thor: Ragnarok     1.0
6                                    Game of Thrones     4.0
7                                            Vikings     4.0
8                                    Boarding School     3.0
DataFrame de valoraciones completas creado:
                                

In [4]:
dfme = df_valoraciones_completas

In [5]:
#generacion de los pesos por genero de cada usuario#
categorias_unicas = set()
for genero in dfre['genres'].dropna().values:
    # Limpiar las categorías eliminando corchetes, comillas y espacios innecesarios
    generos = genero.strip("[]").replace("'", "").replace('"', "").split(", ")
    generos = [g.strip().lower() for g in generos]
    categorias_unicas.update(generos)

categorias_unicas = list(categorias_unicas)

# Crear la matriz de datos para las categorías
datos = []

for row in dfme[dfme["Rating"] > 0]['genres'].dropna().values:
    categorias_peliculas = []
    row_generos = row.strip("[]").replace("'", "").replace('"', "").split(", ")
    row_generos = [g.strip().lower() for g in row_generos]
    for cat in categorias_unicas:
        categorias_peliculas.append(int(cat in row_generos))
    datos.append(categorias_peliculas)

#categorias_peliculas.append(int(cat in row.split(", ")))

df_generos_peliculas = pd.DataFrame(data = datos, columns = list(categorias_unicas))
dfme.reset_index(drop=True, inplace=True)

weighted_genre_matrix = pd.concat([dfme, df_generos_peliculas], axis = 1)
weighted_genre_matrix = (weighted_genre_matrix[categorias_unicas].values.T * weighted_genre_matrix['Rating'].values).T
weighted_genre_matrix = pd.DataFrame(weighted_genre_matrix, columns = categorias_unicas)

usuario_pesos = weighted_genre_matrix.sum()

usuario_pesos = usuario_pesos / usuario_pesos.sum()

for genero in categorias_unicas:
    if genero not in usuario_pesos:
        usuario_pesos[genero] = 0.0000


usuario_pesos

                 0.000000
romance          0.077670
animation        0.000000
documentation    0.029126
fantasy          0.155340
comedy           0.029126
horror           0.029126
war              0.038835
music            0.000000
action           0.223301
history          0.038835
western          0.000000
reality          0.000000
sport            0.000000
european         0.067961
drama            0.165049
crime            0.000000
thriller         0.029126
family           0.019417
scifi            0.097087
dtype: float64

In [6]:

# Definir un umbral de penalización o un factor de penalización
factor_penalizacion = 0.85
umbral_penalizacion = usuario_pesos.sort_values(ascending=False).iloc[2] 

usuario_pesos_penalizados = usuario_pesos.apply(lambda x: (x - umbral_penalizacion) * factor_penalizacion if x < umbral_penalizacion else x)
umbral_penalizacion

0.1553398058252427

In [7]:
usuario_pesos_penalizados

                -0.132039
romance         -0.066019
animation       -0.132039
documentation   -0.107282
fantasy          0.155340
comedy          -0.107282
horror          -0.107282
war             -0.099029
music           -0.132039
action           0.223301
history         -0.099029
western         -0.132039
reality         -0.132039
sport           -0.132039
european        -0.074272
drama            0.165049
crime           -0.132039
thriller        -0.107282
family          -0.115534
scifi           -0.049515
dtype: float64

In [8]:
#creo dataframe columnas por cada genero (en los pesos del usuario) y genero puntuacion por genero
datos = []
for row in dfre['genres'].dropna().values:
    categorias_peliculas = []
    row_generos = row.strip("[]").replace("'", "").replace('"', "").split(", ")
    row_generos = [g.strip().lower() for g in row_generos]
    for cat in categorias_unicas:
        categorias_peliculas.append(int(cat in row_generos))
    datos.append(categorias_peliculas)


df_generos_peliculas = pd.DataFrame(data=datos, columns=list(categorias_unicas))
dfre2 = pd.concat([dfre, df_generos_peliculas], axis=1)
dfre['Puntuacionge'] = (dfre2[categorias_unicas] * usuario_pesos_penalizados).sum(axis=1)

In [9]:
#esto es el codigo que me dijo dmitry lo dejo aqui comentado para luego intentar sustituirlo
#categorias_peliculas.append(int(cat in row.split(", ")))

In [10]:
#generacion de pesos del usuario por director
dfm2=dfme.dropna(subset = ['director_ids'])
dfm2.reset_index(drop=True, inplace=True)
directores = dfm2['director_ids'].unique().tolist()

datos_directores = []

for row in dfm2[dfm2["Rating"] > 0]["director_ids"].values:
    
    c_directores = []
    
    for director in directores :
    

        if director == row:
            c_directores.append(1)  
        else:
            c_directores.append(0)  
    

    datos_directores.append(c_directores)


df_directores = pd.DataFrame(data = datos_directores, columns = list(directores))

weighted_genre_matrix2 = pd.concat([dfm2, df_directores], axis = 1)
weighted_genre_matrix2 = (weighted_genre_matrix2[directores].values.T * weighted_genre_matrix2['Rating'].values).T
weighted_genre_matrix2 = pd.DataFrame(weighted_genre_matrix2, columns = directores )
usuario_d = weighted_genre_matrix2.sum()

usuario_d = usuario_d / usuario_d.sum()

usuario_d

5216, 5215    0.15
1283          0.45
2115          0.10
2354          0.10
20352         0.05
7904          0.15
dtype: float64

In [11]:
factor_penalizacion2 = 1.2
umbral_penalizacion2 = usuario_d.sort_values(ascending=False).iloc[1] 
usuario_dp = usuario_d.apply(lambda x:x * factor_penalizacion2 if x >= umbral_penalizacion2 else x)

In [12]:
# generar dataframe columnas por cada director (en los pesos del usuario) y generar puntuacion por director
datos_directores = []

for row in dfre['director_ids'].values:
    
    c_directores = []
    
    for director in directores :
    

        if director == row:
            c_directores.append(1)  
        else:
            c_directores.append(0)  
    

    datos_directores.append(c_directores)


df_directores = pd.DataFrame(data=datos_directores, columns=list(directores))
dfre3 = pd.concat([dfre, df_directores], axis=1)
dfre["Puntuaciond"]= (dfre3[directores] * usuario_dp).sum(axis=1)

In [13]:
#generacion de pesos del usuario por actor
dfme['actor_ids'] = dfme['actor_ids'].fillna('')
actores_unicos = set()
for actor in dfme['actor_ids'].values:
    actores = actor.split(", ")
    actores_unicos.update(actores)

actores_unicos = list(actores_unicos)



datos = []

for row in dfme[dfme["Rating"] > 0]["actor_ids"].values:

    actores_peliculas = list()
    
    for cat in actores_unicos:
    
        if cat in row.split(", "):
            actores_peliculas.append(1)
        else:
            actores_peliculas.append(0)

    datos.append(actores_peliculas)


df_actores_peliculas = pd.DataFrame(data = datos, columns = list(actores_unicos))
dfme.reset_index(drop=True, inplace=True)

weighted_genre_matrix = pd.concat([dfme, df_actores_peliculas], axis = 1)
weighted_genre_matrix = (weighted_genre_matrix[actores_unicos].values.T * weighted_genre_matrix['Rating'].values).T
weighted_genre_matrix = pd.DataFrame(weighted_genre_matrix, columns = actores_unicos)

usuario_ac = weighted_genre_matrix.sum()

usuario_ac = usuario_ac / usuario_ac.sum()

usuario_ac

81643      0.001866
826256     0.003109
708182     0.001244
1403288    0.005597
126996     0.003109
             ...   
687987     0.000622
49379      0.002488
343075     0.003109
875074     0.001244
2038       0.001244
Length: 464, dtype: float64

In [14]:
# generar dataframe columnas por cada actor (en los pesos del usuario) y generar puntuacion por actor
datos = []
dfre['actor_ids'] = dfre['actor_ids'].fillna('')
for row in dfre['actor_ids'].values:
    actores_peliculas = list()
    for cat in actores_unicos:
        if cat in row.split(", "):
            actores_peliculas.append(1)
        else:
            actores_peliculas.append(0)
    datos.append(actores_peliculas)

df_actores_peliculas = pd.DataFrame(data=datos, columns=list(actores_unicos))
dfre4 = pd.concat([dfre, df_actores_peliculas], axis=1)
dfre["Puntuacionac"]=(dfre4[actores_unicos] * usuario_ac).sum(axis=1)

In [15]:
#lemmatizer = WordNetLemmatizer()
#stop_words = set(stopwords.words('english'))

#def preprocess_text(text):
    #if pd.isnull(text):
       # return ""
    #tokens = word_tokenize(str(text).lower())
    #tokens = [lemmatizer.lemmatize(word) for word in tokens if word.isalnum()]
#     tokens = [word for word in tokens if word not in stop_words]
#     return " ".join(tokens)
    
# # Preprocesa las descripciones
# dfre5 = pd.DataFrame()
# dfre5['processed_description'] = dfre['description'].apply(preprocess_text)

# # Vectoriza las descripciones preprocesadas
# vectorizer = TfidfVectorizer()
# tfidf_matrix = vectorizer.fit_transform(dfre5['processed_description'])

# # Calcula la matriz de similitud de coseno
# similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)

# # Define la función de recomendación
# def get_recommendations(movie_titles, similarity_matrix, df):
#     sim_scores_suma = [0] * len(df)
#     for movie_title in movie_titles:
#         idx = df.index[df['title'] == movie_title].tolist()[0]
#         sim_scores = list(enumerate(similarity_matrix[idx]))
#         for i, score in sim_scores:
#             #if i != idx:  # Omitir la película misma
#              sim_scores_suma[i] += score
    
#     numero_movies = len(movie_titles)
#     sim_scores_media = [score / numero_movies for score in sim_scores_suma]
    

#     for i in range(len(df)):
#         df.at[i, 'Puntuacion_des'] = sim_scores_media[i]
        
#     return df

In [16]:
# if (dfme['Rating'] == 5).any():
#     peliculas_5 = dfme.loc[dfme['Rating'] == 5, 'title'].tolist()
#     if len(peliculas_5) == 1:
#         movie_title = peliculas_5[0]
#         dfre['Puntuacion_des'] = 0
#         dfre['Puntuacion_des'] = dfre['Puntuacion_des'].astype(float)
#         recommendations = get_recommendations([movie_title], similarity_matrix, dfre)
#     else:
#         dfre['Puntuacion_des'] = 0
#         dfre['Puntuacion_des'] = dfre['Puntuacion_des'].astype(float)
#         recommendations = get_recommendations(peliculas_5, similarity_matrix, dfre)
# else:
#     print("No hay ninguna película con un rating de 5.")

In [17]:
dfre = dfre[~dfre['title'].isin(dfme['title'])]
dfre = dfre.copy()

Generacion de recomendaciones


In [18]:
# se normalizan las puntuaciones y se le da un peso de 65 % a genero 20 % a director, 10 a actores y 5 a descripcion
# en caso de no haya valorado ninguna pelicula con un 5 - son 65 % genero 25 % director y 10 % actores


In [19]:
def calcular_puntuacion(row):
    if row["type"] == "MOVIE":
        return (
                0.72 * (row['Puntuacionge'] - dfre['Puntuacionge'].min()) / (dfre['Puntuacionge'].max() - dfre['Puntuacionge'].min()) +
                0.12 * row["Puntuaciond"] / dfre["Puntuaciond"].max() +
                0.08 * row["Puntuacionac"] / dfre["Puntuacionac"].max()+
                0.08 *(row['imdb_score'] - dfre['imdb_score'].min()) / (dfre['imdb_score'].max() - dfre['imdb_score'].min())
            )
    else:
         return (
                0.80 * (row['Puntuacionge'] - dfre['Puntuacionge'].min()) / (dfre['Puntuacionge'].max() - dfre['Puntuacionge'].min()) +
                0.10 * row["Puntuacionac"] / dfre["Puntuacionac"].max()+
                0.10 *(row['imdb_score'] - dfre['imdb_score'].min()) / (dfre['imdb_score'].max() - dfre['imdb_score'].min())
            )

dfre["Puntuacion"] = dfre.apply(calcular_puntuacion, axis=1)

In [20]:
dfrem = dfre[dfre["type"] == "MOVIE"]
dfres = dfre[dfre["type"] == "SHOW"]

In [21]:
recomendaciones1 = dfrem.sort_values(by='Puntuacion', ascending=False)
recomendaciones2 = dfres.sort_values(by='Puntuacion', ascending=False)

In [22]:
recomendaciones1.head(12)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,...,tmdb_score,actors,actor_ids,directors,director_ids,platform,Puntuacionge,Puntuaciond,Puntuacionac,Puntuacion
4403,tm142381,The Lord of the Rings: The Return of the King,MOVIE,Aragorn is revealed as the heir to the ancient...,2003,PG-13,201,"['fantasy', 'action', 'drama']","['NZ', 'US']",NaN,...,8.474,"Elijah Wood, Sean Astin, Ian McKellen, Billy B...","2845, 2857, 2804, 2855, 2846, 1858, 2856, 3260...",Peter Jackson,1283,"HBO, Netflix",0.543689,0.54,0.224502,0.991818
5144,tm158094,The Hobbit: The Battle of the Five Armies,MOVIE,Immediately after the events of The Desolation...,2015,PG-13,144,"['fantasy', 'action']","['US', 'NZ']",NaN,...,7.310,"Ian McKellen, Martin Freeman, Richard Armitage...","2804, 7181, 76404, 2848, 49991, 2694, 20047, 1...",Peter Jackson,1283,HBO,0.378641,0.54,0.061567,0.816999
5739,tm171483,The Hobbit: The Desolation of Smaug,MOVIE,"The Dwarves, Bilbo and Gandalf have successful...",2014,PG-13,161,"['fantasy', 'action']","['US', 'NZ']",NaN,...,7.571,"Ian McKellen, Martin Freeman, Richard Armitage...","2804, 7181, 76404, 1396, 2848, 49991, 20047, 2...",Peter Jackson,1283,HBO,0.378641,0.54,0.049129,0.816203
7275,tm221953,Bāhubali 2: The Conclusion,MOVIE,"When Mahendra, the son of Bāhubali, learns abo...",2017,NaN,167,"['fantasy', 'action', 'drama']",['IN'],NaN,...,7.399,"Prabhas, Rana Daggubati, Sathyaraj, Anushka Sh...","45431, 45432, 45437, 45433, 45436, 45435, 6725...",S. S. Rajamouli,45443,Netflix,0.543689,0.00,0.000000,0.784545
4110,tm137002,Bāhubali: The Beginning,MOVIE,The young Shivudu is left as a foundling in a ...,2015,NaN,158,"['action', 'drama', 'fantasy']",['IN'],NaN,...,7.572,"Prabhas, Rana Daggubati, Tamannaah Bhatia, Anu...","45431, 45432, 45434, 45433, 45435, 45436, 4543...",S. S. Rajamouli,45443,Netflix,0.543689,0.00,0.000000,0.782727
2856,tm1216297,King of Thieves,MOVIE,"Driven by the thirst for revenge, an invincibl...",2022,NaN,108,"['action', 'drama', 'fantasy']",[],NaN,...,NaN,NaN,,NaN,NaN,Amazon,0.543689,0.00,0.000000,0.775455
6054,tm178788,King Kong,MOVIE,"In 1933 New York, an overly ambitious movie pr...",2005,PG-13,187,"['drama', 'action', 'romance']","['DE', 'NZ', 'US']",NaN,...,6.858,"Naomi Watts, Jack Black, Adrien Brody, Thomas ...","1907, 4677, 4612, 1857, 9313, 5360, 1858, 4075...",Peter Jackson,1283,Netflix,0.322330,0.54,0.036692,0.771444
10221,tm374208,Detective Dee: The Four Heavenly Kings,MOVIE,"Dee, the detective serving Chinese empress Wu ...",2018,NaN,132,"['action', 'drama', 'fantasy']","['CN', 'HK']",NaN,...,6.303,"Mark Chao, William Feng, Carina Lau, Lin Gengx...","36788, 36789, 36791, 36575, 200180, 60416, 296...",Tsui Hark,36580,Amazon,0.543689,0.00,0.000000,0.767273
4007,tm134516,"Crouching Tiger, Hidden Dragon: Sword of Destiny",MOVIE,"A story of lost love, young love, a legendary ...",2016,PG-13,96,"['drama', 'fantasy', 'action']",['CN'],NaN,...,6.110,"Michelle Yeoh, Donnie Yen, Jason Scott Lee, Na...","5845, 6663, 6664, 6666, 6665, 6667, 6668, 6669...",Yuen Woo-ping,6676,Netflix,0.543689,0.00,0.002488,0.766341
5476,tm165169,Beowulf & Grendel,MOVIE,The blood-soaked tale of a Norse warrior's bat...,2006,R,103,"['drama', 'fantasy', 'action']","['IS', 'GB', 'US', 'AU', 'CA']",NaN,...,5.698,"Gerard Butler, Spencer Wilding, Stellan Skarsg...","6888, 9046, 1856, 56817, 56818, 56819, 2149, 5...",Sturla Gunnarsson,56827,Amazon,0.543689,0.00,0.002488,0.763614


In [23]:
recomendaciones2.head(12)

,id,title,type,description,release_year,age_certification,runtime,genres,production_countries,seasons,...,tmdb_score,actors,actor_ids,directors,director_ids,platform,Puntuacionge,Puntuaciond,Puntuacionac,Puntuacion
22954,ts89210,Warrior Nun,SHOW,"After waking up in a morgue, an orphaned teen ...",2020,TV-MA,43,"['action', 'drama', 'fantasy']",['US'],2.0,...,8.134,"Alba Baptista, Tristán Ulloa, Kristina Tonteri...","263945, 15911, 1072959, 757594, 124252, 18152,...",NaN,NaN,Netflix,0.543689,0.0,0.000000,0.867045
22687,ts84392,The Mandalorian,SHOW,"After the fall of the Galactic Empire, lawless...",2019,TV-14,40,"['scifi', 'action', 'drama', 'fantasy']",['US'],3.0,...,8.478,"Pedro Pascal, Katee Sackhoff","32436, 27489",NaN,NaN,Disney,0.494175,0.0,0.000000,0.852293
22797,ts86474,Loki,SHOW,After stealing the Tesseract during the events...,2021,TV-14,48,"['drama', 'scifi', 'fantasy', 'action']",['US'],1.0,...,8.170,"Tom Hiddleston, Owen Wilson, Sophia Di Martino...","2134, 4621, 260073, 36136, 14023, 20372",NaN,NaN,Disney,0.494175,0.0,0.000622,0.846888
19273,ts237713,Sweet Tooth,SHOW,On a perilous adventure across a post-apocalyp...,2021,TV-14,46,"['drama', 'scifi', 'fantasy', 'action']",['US'],2.0,...,7.971,"Christian Convery, Nonso Anozie, Adeel Akhtar,...","606757, 13196, 13572, 28926, 16301, 130094, 6061",NaN,NaN,Netflix,0.494175,0.0,0.000000,0.842065
22836,ts87242,Shadow and Bone,SHOW,In a world cleaved in two by a massive barrier...,2021,TV-14,56,"['scifi', 'drama', 'fantasy', 'action']",['US'],2.0,...,8.100,"Jessie Mei Li, Archie Renaux, Freddy Carter, A...","1219860, 989489, 698851, 1080629, 1160365, 7111",NaN,NaN,Netflix,0.494175,0.0,0.000000,0.839793
21203,ts365037,The Bastard Son & the Devil Himself,SHOW,"Caught between two warring clans, the son of a...",2022,TV-MA,49,"['scifi', 'drama', 'fantasy', 'action']",['GB'],1.0,...,7.700,"Jay Lycurgo, Nadia Parkes, Paul Ready, Isobel ...","1390893, 1042587, 114627, 2527516, 1351541",NaN,NaN,Netflix,0.494175,0.0,0.000000,0.838656
20201,ts296563,Who Rules The World,SHOW,A legendary wuxia romance that focuses on the ...,2022,TV-PG,46,"['drama', 'action', 'fantasy', 'romance']",['CN'],1.0,...,7.000,"Yang Yang, Zhao Lusi, Xuan Lu, Lai Yi, Liu Rui...","126992, 898052, 1130646, 835611, 709739, 33418...",Yin Tao,649537,Netflix,0.477670,0.0,0.000000,0.836390
21975,ts57325,The Wheel of Time,SHOW,"Follow Moiraine, a member of the shadowy and i...",2021,TV-MA,59,"['scifi', 'drama', 'fantasy', 'action']",['US'],1.0,...,7.766,"Rosamund Pike, Daniel Henney, Zoe Robins, Barn...","9044, 24694, 606259, 226803, 121886, 737756, 8...",NaN,NaN,Amazon,0.494175,0.0,0.000000,0.835247
20709,ts325752,The Lord of the Rings: The Rings of Power,SHOW,Epic drama set thousands of years before the e...,2022,TV-14,70,"['fantasy', 'action', 'drama', 'scifi']",['US'],1.0,...,7.491,"Morfydd Clark, Robert Aramayo, Charlie Vickers...","62006, 227043, 1112685, 12491, 36942, 1309559,...",NaN,NaN,Amazon,0.494175,0.0,0.000000,0.831838
19814,ts271888,Fate: The Winx Saga,SHOW,The coming-of-age journey of five fairies atte...,2021,NaN,51,"['scifi', 'drama', 'fantasy', 'action']",['GB'],2.0,...,8.283,"Abigail Cowen, Hannah van der Westhuysen, Prec...","860273, 331808, 1156312, 239999, 1275303, 1052...",NaN,NaN,Netflix,0.494175,0.0,0.000000,0.830702
